Chama os cachorros

In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

Using TensorFlow backend.


There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [2]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

Pré-processamento

In [3]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|███████████████████████████████████████| 836/836 [00:04<00:00, 173.15it/s]


REDE CNN

In [4]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential

model = Sequential()

### TODO: Define your architecture.

num_classes = 133 #Quantidade de raças de cachorros

#Convolutional layer configuration
filters = 64
kernel_size = (3, 3)
strides = (2, 2)
input_shape = (224, 224, 3)
pool_size = (2, 2)

model.add(Conv2D(filters, kernel_size, input_shape=input_shape, padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size, strides))
model.add(Activation('relu'))


filters = 32
kernel_size = (2, 2)
strides = (2, 2)
pool_size = (2, 2)

model.add(Conv2D(filters, kernel_size, input_shape=input_shape, padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size, strides))
model.add(Activation('relu'))

'''
#filters = 64
#kernel_size = (2, 2)
#strides = (2, 2)
pool_size = (2, 2)

model.add(Conv2D(filters, kernel_size, input_shape=input_shape, padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size, strides))
model.add(Activation('relu'))
'''

#model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dropout(0.2))


#Dense layer
#model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.5))

#Output layer
model.add(Dense(num_classes, activation='softmax', name='output'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 64)      1792      
_________________________________________________________________
activation_1 (Activation)    (None, 222, 222, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 64)      0         
_________________________________________________________________
activation_2 (Activation)    (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 110, 110, 32)      8224      
_________________________________________________________________
activation_3 (Activation)    (None, 110, 110, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 55, 55, 32)        0         
__________

Compilar modelo

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=5, batch_size=20, verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/5
  60/6680 [..............................] - ETA: 508s - loss: 4.8859 - acc: 0.0000e+0 - ETA: 278s - loss: 7.8031 - acc: 0.0000e+0 - ETA: 201s - loss: 9.3446 - acc: 0.0000e+00

ResourceExhaustedError: OOM when allocating tensor with shape[20,64,111,111]
	 [[Node: gradients/max_pooling2d_1/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, _class=["loc:@max_pooling2d_1/MaxPool"], data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1], _device="/job:localhost/replica:0/task:0/gpu:0"](activation_1/Relu, max_pooling2d_1/MaxPool, gradients/activation_2/Relu_grad/ReluGrad)]]

Caused by op 'gradients/max_pooling2d_1/MaxPool_grad/MaxPoolGrad', defined at:
  File "C:\ProgramData\Anaconda3\envs\dog-project\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\dog-project\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\dog-project\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\envs\dog-project\lib\site-packages\traitlets\config\application.py", line 658, in la

In [ ]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)